In [746]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ISLP.models import ModelSpec as MS
import statsmodels.api as sm
import keras
from keras import layers
from keras import ops
from sklearn.model_selection import train_test_split
from ISLP import confusion_table

In [747]:
import os
PathToRepo = os.path.normpath(os.getcwd() + os.sep + os.pardir)
Df = pd.read_csv(PathToRepo + '\\Data\\Faellesdata_cleaned.csv')

x = Df.drop('IsMigratorInt', axis=1) 
for i in range(0, x.shape[1]): #Standardizing the data
    x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

x_train, x_test, y_train, y_test = train_test_split(x, Df['IsMigratorInt'], test_size=0.2, random_state=42)

In [748]:
x_train = MS(x_train, intercept = False).fit_transform(x_train)

In [749]:
modelnn = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [750]:
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7796 - loss: 0.4602
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8483 - loss: 0.3311
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8458 - loss: 0.3104
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - accuracy: 0.8524 - loss: 0.2969
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8660 - loss: 0.2753
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8691 - loss: 0.2722
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8678 - loss: 0.2747
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8726 - loss: 0.2627  
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8731 - loss: 0.2601
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8767 - loss: 0.2573  
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - accuracy: 0.8761 - loss: 0.2537
Epoch 12/100
250/250 ━━━━━━━━━━━━

In [751]:
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8622 - loss: 0.3352


[0.34855884313583374, 0.8544999957084656]

In [752]:
from sklearn.metrics import accuracy_score

In [753]:
confusion_bag = confusion_table(modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5, y_test)
confusion_bag.to_latex(PathToRepo + '\\Tables\\Confusion1.tex', index_names = True, caption="Confusion table for NN1", label="tab:NN1", position="h!")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [754]:

Models = []
modelconfig = modelnn.get_config()

for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 1', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layer #': i, 'Layer Type': layer_type, 
                   'Neurons': neurons, 
                   'Activation': activation,
                   'epochs': 100})

ModelsDf = pd.DataFrame(Models)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [755]:
modelsoftmax = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [756]:
modelsoftmax.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelsoftmax.fit(x_train, keras.utils.to_categorical(y_train), epochs=100)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8260 - loss: 0.4144
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8528 - loss: 0.3231
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8517 - loss: 0.3041
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8647 - loss: 0.2805
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8607 - loss: 0.2862
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8705 - loss: 0.2700
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8711 - loss: 0.2690
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8747 - loss: 0.2623
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8762 - loss: 0.2555
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8787 - loss: 0.2484
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8784 - loss: 0.2540
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━

In [757]:
confusion_bag = confusion_table(np.argmax(modelsoftmax.predict(MS(x_test, intercept = False).fit_transform(x_test)), axis = 1)    , y_test)
confusion_bag.to_latex(PathToRepo + '\\Tables\\Confusion2.tex', index_names = True, caption="Confusion table for NN2", label="tab:NN2", position="h!")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


In [758]:
Models = []
modelnn = modelsoftmax

modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 2'
                   , 'Optimizer': modelnn.optimizer.get_config()['name']
                   , 'Loss': modelnn.loss
                    , 'Accuracy': accuracy_score(y_test, np.argmax(modelsoftmax.predict(MS(x_test, intercept = False).fit_transform(x_test)), axis=1))
                   , 'Layer #': i, 'Layer Type': layer_type, 'Neurons': neurons
                   , 'Activation': activation,
                   'epochs': 100})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layer #', 'Layer Type''Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy', 'epochs'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


KeyError: 'Layer TypeNeurons'

In [ ]:
modelnn = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(1, activation='sigmoid')
])
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100, verbose = 0)
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)
modelconfig = modelnn.get_config()



C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.8655 - loss: 0.2635


In [ ]:
confusion_bag = confusion_table(modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5, y_test)
confusion_bag.to_latex(PathToRepo + '\\Tables\\Confusion3.tex', index_names = True, caption="Confusion table for NN3", label="tab:NN3", position="h!")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step


In [ ]:
Models = []
modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 3', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layer #': i, 'Layer Type': layer_type, 
                   'Neurons': neurons, 
                   'Activation': activation,
                   'epochs': 100})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layer #', 'Layer Type''Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy', 'epochs'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step


In [ ]:
modelnn = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100, verbose = 0)
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8406 - loss: 0.3061  


[0.30281969904899597, 0.8424999713897705]

In [ ]:
confusion_bag = confusion_table(modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5, y_test)
confusion_bag.to_latex(PathToRepo + '\\Tables\\Confusion4.tex', index_names = True, caption="Confusion table for NN4", label="tab:NN4", position="h!")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [ ]:
Models = []
modelconfig = modelnn.get_config()
modelconfig['layers'][2]
for i in range(1, len(modelconfig['layers'])):
    layer_type = modelconfig['layers'][i]['class_name']
    if layer_type == 'Dropout':
        rate = modelconfig['layers'][i]['config']['rate']
        Models.append({'Model': 'Neural Network 4', 
                       'Optimizer': modelnn.optimizer.get_config()['name'], 
                       'Loss': modelnn.loss, 
                       'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                       'Layer #': i, 'Layer Type': layer_type, 
                       'Rate': rate,
                        'epochs': 100})
        continue
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    Models.append({'Model': 'Neural Network 4', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layer #': i, 'Layer Type': layer_type, 
                   'Neurons': neurons, 
                   'Activation': activation,
                   'epochs': 100})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layer #', 'Layer Type''Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy', 'epochs'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step


Note That the full data file is not included in the repository, as it is too large and that the code below takes over 30 mins to run. The code above just loads the model and calculates the accuracy score.

In [ ]:
# from astropy.io import ascii
# FullData = ascii.read(PathToRepo + '\\Data\\m12i_res7100_mhdcv.disk.ecsv', guess = False)
# FullData = FullData.to_pandas()

In [ ]:
# FullData['IsMigrator'] = abs(FullData['Rcyl'] - FullData['Rcyl_form']) > 1.5
# FullData['IsMigratorInt'] = FullData['IsMigrator'].astype(int)
# FullData = FullData.drop(['rsph', 'x', 'y',  'vx', 'vy', 
#        'rsph_form', 'x_form', 'y_form', 'z_form', 'vx_form', 'vy_form',
#        'vz_form', 'Rcyl_form', 'phi_form', 'vRcyl_form', 'vphi_form', "IsMigrator"], axis = 1)

In [ ]:
# Df = FullData
# x = Df.drop('IsMigratorInt', axis=1) 
# for i in range(0, x.shape[1]): #Standardizing the data
#     x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

# x_train_full, x_test_full, y_train_full, y_test_full = train_test_split(x, Df['IsMigratorInt'], test_size=0.2, random_state=42)

In [ ]:
# modelnn = keras.Sequential([
#     layers.Dense(16, activation='relu', input_shape=[x_train_full.shape[1]]),
#     layers.Dense(1, activation='sigmoid')
# ])
# modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# modelnn.fit(x_train_full, y_train_full, epochs=10)

In [ ]:
Models = []
modelnn = keras.models.load_model('modelnn.keras')

modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 5', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layer #': i, 'Layer Type': layer_type, 
                   'Neurons': neurons, 
                   'Activation': activation,
                   'epochs': 10})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layer #', 'Layer Type''Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy', 'epochs'])
confusion_bag = confusion_table(modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5, y_test)
confusion_bag.to_latex(PathToRepo + '\\Tables\\Confusion5.tex', index_names = True, caption="Confusion table for NN5", label="tab:NN5", position="h!")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step


C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step


In [ ]:
ModelsDf
ModelsDf.index = ModelsDf['Model']
ModelsDf = ModelsDf.drop('Model', axis = 1)

In [ ]:
ModelsDf.index = ModelsDf.index.rename('Model')

In [ ]:
ModelsDf.index = pd.MultiIndex.from_tuples([(ModelsDf.index[i],i) for i in range(0, ModelsDf.shape[0])])

In [ ]:
ModelsDf['Loss'].replace("_", "\\textunderscore ", regex=True, inplace=True)

C:\Users\lorentsen\AppData\Local\Temp\ipykernel_51116\2191194268.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ModelsDf['Loss'].replace("_", "\\textunderscore ", regex=True, inplace=True)


In [ ]:
ModelsDf.to_latex(PathToRepo + '\\Tables\\Models.tex', index_names = True, na_rep='-', multirow = True, float_format="%.3f", caption="Differnet neural networks", label="tab:RawData", position="h!")
ModelsDf

Optimizer                                     Loss  \
Neural Network 1 0       adam       binary\textunderscore crossentropy   
                 1       adam       binary\textunderscore crossentropy   
                 2       adam       binary\textunderscore crossentropy   
Neural Network 2 3       adam  categorical\textunderscore crossentropy   
                 4       adam  categorical\textunderscore crossentropy   
                 5       adam  categorical\textunderscore crossentropy   
Neural Network 3 6       adam       binary\textunderscore crossentropy   
                 7       adam       binary\textunderscore crossentropy   
Neural Network 4 8       adam       binary\textunderscore crossentropy   
                 9       adam       binary\textunderscore crossentropy   
                 10      adam       binary\textunderscore crossentropy   
                 11      adam       binary\textunderscore crossentropy   
                 12      adam       binary\textunderscore crossentropy   
Neural Network 5 13   rmsprop       binary\textunderscore crossentropy   
                 14   rmsprop       binary\textunderscore crossentropy   

                     Accuracy   Layers  Neurons Activation  epochs  Rate  
Neural Network 1 0     0.8510    Dense      1.0    sigmoid     100   NaN  
                 1     0.8510    Dense     64.0       relu     100   NaN  
                 2     0.8510    Dense     64.0       relu     100   NaN  
Neural Network 2 3     0.8515    Dense      2.0    softmax     100   NaN  
                 4     0.8515    Dense     64.0       relu     100   NaN  
                 5     0.8515    Dense     64.0       relu     100   NaN  
Neural Network 3 6     0.8655    Dense      1.0    sigmoid     100   NaN  
                 7     0.8655    Dense     16.0       relu     100   NaN  
Neural Network 4 8     0.8425    Dense      1.0    sigmoid     100   NaN  
                 9     0.8425    Dense     16.0       relu     100   NaN  
                 10    0.8425    Dense     16.0       relu     100   NaN  
                 11    0.8425  Dropout      NaN        NaN     100   0.5  
                 12    0.8425  Dropout      NaN        NaN     100   0.5  
Neural Network 5 13    0.8800    Dense      1.0    sigmoid      10   NaN  
                 14    0.8800    Dense     16.0       relu      10   NaN